In [ ]:
from pathlib import Path
import pandas as pd
import altair as alt
from scipy.stats.mstats import gmean
import numpy as np
import re
import collections

In [ ]:
# Options.

reload_data = False
make_plots = False
make_tables = True

In [ ]:
# Folders.

stats_dir = Path('./stats')
plot_dir = Path('./plots')
table_dir = Path('./tables')

if make_plots:
    plot_dir.mkdir(parents=True, exist_ok=True)
if make_tables:
    table_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Combinations of width configuration mode and partial clock gating on/off.

power_modes = {'power':False, 'power_clkgated':True}

modes = {
    'Original'      :{'Fuse':False, 'Penalty':False, 'Gated':False},
    'Packing'       :{'Fuse':False, 'Penalty':False, 'Gated':True},
    'PackingPenalty':{'Fuse':False, 'Penalty':True, 'Gated':True},
    'Fusing'        :{'Fuse':True, 'Penalty':False, 'Gated':True},
    'FusingPenalty' :{'Fuse':True, 'Penalty':True, 'Gated':True}
}
modes_order = {'Original':0, 'Packing':1, 'PackingPenalty':2, 'Fusing':3, 'FusingPenalty':4}
modes_nopenalty = ['Original', 'Packing', 'Fusing']
modes_penalty = ['Original', 'PackingPenalty', 'FusingPenalty']
modes_mechanisms = ['Packing', 'Fusing', 'PackingPenalty', 'FusingPenalty']

df_modes = pd.DataFrame.from_dict(modes, orient='index') 
df_modes = df_modes.rename_axis('Mode').reset_index()


# Reference config for each CPU.

cpus     = ['A76', 'HP']
ref_mode = 'Original'
ref_fu   = {'A76':2, 'HP':3}

In [ ]:
# Parameter filtering.

component = 'fp/simd alu'

In [ ]:
# Plot labels.

metric_labels = {'Dynamic':'Dynamic Energy Reduction [%]',
                 'Leakage':'Leakage Energy Reduction [%]',
                 'Total':'Total Energy Reduction [%]',
                 'Time':'Normalized Execution Time'}

In [ ]:
# Benchmarks to include, and their name in the plot.

benchmarks = {
    #'hmmer-small_1':'hmmer',
    #'fft_simsmall_1':'fft-splash',
    'integerNN_1':'intNN',
    'streamvbyte_1':'streamvbyte',
    'img_cartoon_tiny_image1_50per':'cartoon',
    'img_cartoon_tiny_image2_50per':'cartoon',
    'img_cartoon_tiny_image3_50per':'cartoon',
    'img_cartoon_tiny_image4_50per':'cartoon',
    'img_cartoon_tiny_image5_50per':'cartoon',
    'img_canny_tiny_image1_50per':'canny',
    'img_canny_tiny_image2_50per':'canny',
    'img_canny_tiny_image3_50per':'canny',
    'img_canny_tiny_image4_50per':'canny',
    'img_canny_tiny_image5_50per':'canny',
    'img_hist_tiny_image1_50per':'hist',
    'img_hist_tiny_image2_50per':'hist',
    'img_hist_tiny_image3_50per':'hist',
    'img_hist_tiny_image4_50per':'hist',
    'img_hist_tiny_image5_50per':'hist',
    'img_integral_tiny_image1_50per':'img_integral',
    'img_integral_tiny_image2_50per':'img_integral',
    'img_integral_tiny_image3_50per':'img_integral',
    'img_integral_tiny_image4_50per':'img_integral',
    'img_integral_tiny_image5_50per':'img_integral',
    'conv_tiny_image1_50per':'conv',
    'conv_tiny_image2_50per':'conv',
    'conv_tiny_image3_50per':'conv',
    'conv_tiny_image4_50per':'conv',
    'conv_tiny_image5_50per':'conv',
    'img_erode_tiny_image1_50per':'erode',
    'img_erode_tiny_image2_50per':'erode',
    'img_erode_tiny_image3_50per':'erode',
    'img_erode_tiny_image4_50per':'erode',
    'img_erode_tiny_image5_50per':'erode',
    'img_median_tiny_image1_50per':'median',
    'img_median_tiny_image2_50per':'median',
    'img_median_tiny_image3_50per':'median',
    'img_median_tiny_image4_50per':'median',
    'img_median_tiny_image5_50per':'median',
    'amax_cols_tiny_16bit_0':'amax',
    'amax_cols_tiny_16bit_1':'amax',
    'amax_cols_tiny_16bit_2':'amax',
    'amax_cols_tiny_16bit_3':'amax',
    'amax_cols_tiny_16bit_4':'amax',
    #'asum_cols_tiny_16bit_0':'asum',
    #'asum_cols_tiny_16bit_1':'asum',
    #'asum_cols_tiny_16bit_2':'asum',
    #'asum_cols_tiny_16bit_3':'asum',
    #'asum_cols_tiny_16bit_4':'asum',
    'gemv_tiny_16bit_0':'gemv',
    'gemv_tiny_16bit_1':'gemv',
    'gemv_tiny_16bit_2':'gemv',
    'gemv_tiny_16bit_3':'gemv',
    'gemv_tiny_16bit_4':'gemv',
    #'ger_tiny_16bit_0':'ger',
    #'ger_tiny_16bit_1':'ger',
    #'ger_tiny_16bit_2':'ger',
    #'ger_tiny_16bit_3':'ger',
    #'ger_tiny_16bit_4':'ger',
    'sqnrm2_cols_tiny_16bit_0':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_1':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_2':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_3':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_4':'sqnrm2',
    'gemm_tiny_16bit_0':'gemm',
    'gemm_tiny_16bit_1':'gemm',
    'gemm_tiny_16bit_2':'gemm',
    'gemm_tiny_16bit_3':'gemm',
    'gemm_tiny_16bit_4':'gemm',
    #'fft_tiny_16bit_0':'fft',
    #'fft_tiny_16bit_1':'fft',
    #'fft_tiny_16bit_2':'fft',
    #'fft_tiny_16bit_3':'fft',
    #'fft_tiny_16bit_4':'fft',
    'fft_tiny_12bit_0':'fft',
    'fft_tiny_12bit_1':'fft',
    'fft_tiny_12bit_2':'fft',
    'fft_tiny_12bit_3':'fft',
    'fft_tiny_12bit_4':'fft',
    #'fft_tiny_8bit_0':'fft',
    #'fft_tiny_8bit_1':'fft',
    #'fft_tiny_8bit_2':'fft',
    #'fft_tiny_8bit_3':'fft',
    #'fft_tiny_8bit_4':'fft'
}

bench_thesis = {'intNN':'A1', 'streamvbyte':'A2', 'cartoon':'A3', 'sqnrm2':'K1', 'amax':'K2', 'gemv':'K3', 'gemm':'K4', 'fft':'K5', 'conv':'K6', 
                'median':'K7', 'img_integral':'K8', 'hist':'K9', 'canny':'K10', 'erode':'K11'}
bench_paper = {'intNN':'A1', 'streamvbyte':'A2', 'cartoon':'A3', 'sqnrm2':'K1', 'amax':'K2', 'gemm':'K3', 'fft':'K4', 'conv':'K5',
               'median':'K6', 'img_integral':'K7', 'canny':'K8', 'erode':'K9'}

In [ ]:
# Create table with normalized energy and time results for the specified runs.

if reload_data:
    # Reload data from the stats folder.

    # List all the run output folder paths.
    runs = list(Path(stats_dir).glob('**/roi.txt'))
    runs = [r.parents[0] for r in runs]

    # Load relevant stats to a large dataframe.
    df = []
    for r in runs:
        # Params that can only be obtained from the path.
        cpu = r.parents[1].name.split('.')[0]
        pen = int(re.findall('\d?(?=pen)', r.parents[1].name)[0]) > 0
        
        # Params that can be obtained from config.csv.
        try:
            param = pd.read_csv(r/'config.csv', header=0)
        except:
            continue
        fu = param.iloc[0]['simdCount']
        fuse = param.iloc[0]['simdFuseCap'] > 0
        wBlock = int(param.iloc[0]['widthBlockSize'])
        bench = Path(param.iloc[0]['bootscript']).stem
        
        if cpu in cpus and bench in benchmarks.keys():
            # Time results.
            time = pd.read_csv(r/'time.csv', header=0)
            try:
                time = float(time.iloc[0]['sim seconds'])
            except TypeError:
                continue
            except IndexError:
                continue

            # Mcpat results. (FIXME: Can change here for more components!)
            for stem, gated in power_modes.items():
                mcpat = pd.read_csv(r/'{}.csv'.format(stem))
                mcpat = mcpat[mcpat['Component'] == component]

                dynamic = float(mcpat.loc[:, 'Runtime Dynamic [W]'].iloc[0] * time)
                leakage = float((mcpat.loc[:, ['Gate Leakage [W]', 'Subthreshold Leakage [W]']].sum(axis='columns')).iloc[0]) * time

                # Add entry to the table.
                df.append({'Benchmark':bench, 'CPU':cpu, 'SIMD FU':fu, 'Fuse':fuse, 'Penalty':pen, 'Width Block':wBlock,
                        'Gated':gated, 'Time':time, 'Leakage':leakage, 'Dynamic':dynamic, 'Total':dynamic+leakage})
        
    df = pd.DataFrame(df)
    df = df.merge(df_modes, how='inner', on=['Fuse', 'Penalty', 'Gated'])
    df = df.drop(columns=['Fuse', 'Penalty', 'Gated'])

    # Calculate normalized metrics for each run.
    df_norm = df.copy()
    df_norm['Ref SIMD FU'] = df_norm['CPU'].replace(ref_fu)
    df_norm['Ref Mode'] = ref_mode
    df_norm = df_norm.merge(df, left_on=['Benchmark', 'CPU', 'Width Block', 'Ref SIMD FU', 'Ref Mode'],
                            right_on=['Benchmark', 'CPU', 'Width Block', 'SIMD FU', 'Mode'],
                            suffixes=('', ' ref'))
    df_norm['Time'] = df_norm['Time'] / df_norm['Time ref']
    df_norm['Dynamic']   = df_norm['Dynamic'] / df_norm['Dynamic ref']
    df_norm['Leakage']   = df_norm['Leakage'] / df_norm['Leakage ref']
    df_norm['Total']     = df_norm['Total'] / df_norm['Total ref']
    df_norm = df_norm.drop(columns=['Ref SIMD FU', 'Ref Mode', 'SIMD FU ref', 'Mode ref', 'Time ref',
                                    'Leakage ref', 'Dynamic ref', 'Total ref'])
    df_norm = pd.melt(df_norm, id_vars=['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block'],
                    var_name='Metric', value_name='Value').copy()
    df_norm = df_norm.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])
    df_norm

    # Apply geometric mean to reduce several runs of the same benchmark.
    df_norm = df_norm.groupby(by=[benchmarks, None, None, None, None, None],
                            level=['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])['Value'].apply(lambda group: gmean(group).mean()).reset_index().set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])

    # Save dataframe.
    if make_tables:
        df_norm.reset_index().set_index(['CPU', 'SIMD FU', 'Mode', 
                                        'Width Block', 'Benchmark',
                                        'Metric']).sort_index().to_csv(table_dir / 'results_diffenergy_time.csv')
else:
    # Use data already generated.

    df_norm = pd.read_csv(table_dir / 'results_diffenergy_time.csv', header=0,
                          index_col=['Benchmark', 'CPU', 'SIMD FU', 'Mode',
                                     'Width Block', 'Metric'])
    df_norm = df_norm.sort_index()

In [ ]:
df_norm

In [ ]:
# Create 'average' benchmark using the geometric mean.
average = df_norm.loc(axis=0)[list(bench_thesis.keys())].groupby(['CPU', 'SIMD FU', 'Mode',
                                                                  'Width Block', 'Metric'], as_index=True, sort=False)['Value'].apply(lambda group: gmean(group).mean()).reset_index()
average['Benchmark'] = 'average'
average = average.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])
average_paper = df_norm.loc(axis=0)[list(bench_paper.keys())].groupby(['CPU', 'SIMD FU', 'Mode',
                                                                       'Width Block', 'Metric'], as_index=True, sort=False)['Value'].apply(lambda group: gmean(group).mean()).reset_index()
average_paper['Benchmark'] = 'average-paper'
average_paper = average_paper.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])

df_energy_time = pd.concat([df_norm, average, average_paper])

# Calculate reduction % for energy metrics.  
df_energy_time.loc(axis=0)[pd.IndexSlice[:,:,:,:,:,['Dynamic', 'Leakage', 'Total']]] = \
    100 * (1 - df_energy_time.loc(axis=0)[pd.IndexSlice[:,:,:,:,:,['Dynamic', 'Leakage', 'Total']]])

df_energy_time

In [ ]:
# Help function for plots.

def filter_data(df, bench=None, cpu=None, metric=None, wBlock=None, fu=None, mode=None):
    if bench is not None:
        df = df[df['Benchmark'].isin(bench)]
    if cpu is not None:
        df = df[df['CPU'].isin(cpu)]
    if metric is not None:
        df = df[df['Metric'] == metric]
    if wBlock is not None:
        df = df[df['Width Block'].isin(wBlock)]
    if fu is not None:
        if type(fu) is list:
            df = df[df['SIMD FU'].isin(fu)]
        elif type(fu) is dict:
            df = df[df['SIMD FU'] == df['CPU'].replace(fu)]
    if mode is not None:
        df = df[df['Mode'].isin(mode)]
    
    return df


def get_chart(df, bench, view='benchmark', cpu=None, metric='Time', wBlock=None, fu=None, mode=None, domain=None, 
              width=800, height=200, label='Configuration', label_order=None, scheme='category', labelXAngle=0):
    if view not in ['benchmark', 'wblock']:
        raise ValueError('Unsupported "view" mode.')
    
    if metric not in metric_labels:
        raise ValueError('Unsupported "metric"')

    if not isinstance(bench, dict):
        if isinstance(bench, collections.Iterable):
            bench = {b:b for b in bench}
        else:
            raise ValueError('"bench" must be a dictionary or an iterable')

    df_metric = filter_data(df, bench=list(bench.keys()), cpu=cpu, metric=metric, wBlock=wBlock, fu=fu, mode=mode)
        
    df_metric['Metric'] = df_metric['Metric'].replace(metric_labels)
    
    ct = alt.Chart(df_metric).mark_line(point=True)
    
    if view == 'benchmark':
        df_metric['Benchmark'] = df_metric['Benchmark'].replace(bench)

        ct = ct.encode(
            x=alt.X(
                'Benchmark:N',
                title="",
                axis=alt.Axis(labelAngle=labelXAngle),
                sort=list(bench.values())
            ),
            y=alt.Y(
                'Value:Q',
                scale=alt.Scale(zero=False) if domain is None else alt.Scale(domain=domain),
                title=metric_labels[metric]
            ),
            color=alt.Color(
                'Label:N',
                scale=alt.Scale(range=scheme),
                sort=label_order,
                title=label
            ),
            shape=alt.Shape(
                'Mode:N',
                scale=alt.Scale(range=['circle', 'diamond', 'cross']),
                sort=list(modes.keys()),
                legend=alt.Legend(symbolSize=160)
            ),
            strokeDash=alt.StrokeDash(
                'Mode:N',
                sort=list(modes.keys())
            )
        )
    else:
        ct = ct.encode(
            x=alt.X(
                'Width Block:O',
                title="Width Block [bits]",
                axis=alt.Axis(labelAngle=0),
                sort='ascending'
            ),
            y=alt.Y(
                'Value:Q',
                scale=alt.Scale(zero=False) if domain is None else alt.Scale(domain=domain),
                title=metric_labels[metric]
            ),
            color=alt.Color(
                'Label:N',
                scale=alt.Scale(range=scheme),
                sort=label_order,
                title=label
            )
        )

    ct = ct.properties(
        height=height,
        width=width
    )
    
    return ct


def configure_chart(ct, orient='right'):
    ct = ct.configure_axis(
        labelFontSize=15,
        titleFontSize=16,
        grid=True
    ).configure_header(
        labelFontSize=15,
        titleFontSize=18
    ).configure_legend(
        orient=orient,
        labelLimit=0,
        titleFontSize=18,
        labelFontSize=16,
        symbolType='stroke',
        symbolSize=800,
        symbolStrokeWidth=5
    ).configure_line(
        size=2.5
    ).configure_point(
        size=120
    )
    return ct

In [ ]:
# Add config column.

df_plot = df_energy_time.reset_index().copy()
df_plot['Label'] = df_plot['CPU'] + '-' + df_plot['Mode'] + '-' + df_plot['SIMD FU'].astype(str) + 'FU'
df_plot['Label Order'] = df_plot['Mode'].replace(modes_order) * df_plot['SIMD FU'].max() + (df_plot['SIMD FU'].max()-df_plot['SIMD FU'])
df_plot

In [ ]:
# Get configuration order.

label_order = list(df_plot.sort_values(['CPU', 'Label Order'])['Label'].unique())
label_order

In [ ]:
# Scales.

time_scale = (0, 3)
half_time_scale = (0, 1.5)
energy_scale = (-20, 70)

In [ ]:
# Plots A76, no penalty.

labelXAngle = -30
bench = list(bench_thesis.keys())

scheme = ['#ffaa73', '#a84400',
          '#95baff', '#005dff',
          '#84cd84', '#007700']
ct_nopen_A76 = get_chart(df_plot, bench, cpu=['A76'], metric='Dynamic', wBlock=[8], mode=modes_nopenalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
               get_chart(df_plot, bench, cpu=['A76'], metric='Leakage', wBlock=[8], mode=modes_nopenalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
               get_chart(df_plot, bench, cpu=['A76'], metric='Total', wBlock=[8], mode=modes_nopenalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
               get_chart(df_plot, bench, cpu=['A76'], metric='Time', wBlock=[8], mode=modes_nopenalty, domain=time_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle)
ct_nopen_A76 = configure_chart(ct_nopen_A76)
ct_nopen_A76

In [ ]:
# Plots A76, penalty.

labelXAngle = -30
bench = list(bench_thesis.keys())

scheme = ['#ffaa73', '#a84400',
          '#95baff', '#005dff',
          '#84cd84', '#007700']
ct_pen_A76 = get_chart(df_plot, bench, cpu=['A76'], metric='Dynamic', wBlock=[8], mode=modes_penalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
             get_chart(df_plot, bench, cpu=['A76'], metric='Leakage', wBlock=[8], mode=modes_penalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
             get_chart(df_plot, bench, cpu=['A76'], metric='Total', wBlock=[8], mode=modes_penalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
             get_chart(df_plot, bench, cpu=['A76'], metric='Time', wBlock=[8], mode=modes_penalty, domain=time_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle)
ct_pen_A76 = configure_chart(ct_pen_A76)
#ct_pen_A76

In [ ]:
# Plots HP, no penalty.

labelXAngle = -30
bench = list(bench_thesis.keys())

scheme = ['#ffc9a5', '#ff8533', '#8c3800',
          '#c5d2e9', '#5ea1ff', '#004cd2',
          '#9ae59a', '#3abf3a', '#006000']
ct_nopen_HP = get_chart(df_plot, bench, cpu=['HP'], metric='Dynamic', wBlock=[8], mode=modes_nopenalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
              get_chart(df_plot, bench, cpu=['HP'], metric='Leakage', wBlock=[8], mode=modes_nopenalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
              get_chart(df_plot, bench, cpu=['HP'], metric='Total', wBlock=[8], mode=modes_nopenalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
              get_chart(df_plot, bench, cpu=['HP'], metric='Time', wBlock=[8], mode=modes_nopenalty, domain=time_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle)
ct_nopen_HP = configure_chart(ct_nopen_HP)
ct_nopen_HP

In [ ]:
# Plots HP, penalty.

labelXAngle = -30
bench = list(bench_thesis.keys())

scheme = ['#ffc9a5', '#ff8533', '#8c3800',
          '#c5d2e9', '#5ea1ff', '#004cd2',
          '#9ae59a', '#3abf3a', '#006000']
ct_pen_HP = get_chart(df_plot, bench, cpu=['HP'], metric='Dynamic', wBlock=[8], mode=modes_penalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
            get_chart(df_plot, bench, cpu=['HP'], metric='Leakage', wBlock=[8], mode=modes_penalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
            get_chart(df_plot, bench, cpu=['HP'], metric='Total', wBlock=[8], mode=modes_penalty, domain=energy_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
            get_chart(df_plot, bench, cpu=['HP'], metric='Time', wBlock=[8], mode=modes_penalty, domain=time_scale, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle)
ct_pen_HP = configure_chart(ct_pen_HP)
#ct_pen_HP

In [ ]:
# Plots thesis.

scheme_a76 = ['#ffaa73', '#a84400', '#95baff', '#005dff', '#84cd84', '#007700']
scheme_hp = ['#ffc9a5', '#ff8533', '#8c3800', '#c5d2e9', '#5ea1ff', '#004cd2', '#9ae59a', '#3abf3a', '#006000']
scheme_join = scheme_a76 + scheme_hp
half_width = 400

ct_thesis_a76_nopen = get_chart(df_plot, bench_thesis, cpu=['A76'],metric='Dynamic', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                                width=half_width, label_order=label_order, scheme=scheme_join) & \
                      get_chart(df_plot, bench_thesis, cpu=['A76'], metric='Leakage', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                                width=half_width, label_order=label_order, scheme=scheme_join) & \
                      get_chart(df_plot, bench_thesis, cpu=['A76'], metric='Total', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                                width=half_width, label_order=label_order, scheme=scheme_join) & \
                      get_chart(df_plot, bench_thesis, cpu=['A76'], metric='Time', wBlock=[8], mode=modes_nopenalty, domain=time_scale,
                                width=half_width, label_order=label_order, scheme=scheme_join)
ct_thesis_hp_nopen = get_chart(df_plot, bench_thesis, cpu=['HP'], metric='Dynamic', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                               width=half_width, label_order=label_order, scheme=scheme_join) & \
                     get_chart(df_plot, bench_thesis, cpu=['HP'], metric='Leakage', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                               width=half_width, label_order=label_order, scheme=scheme_join) & \
                     get_chart(df_plot, bench_thesis, cpu=['HP'], metric='Total', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                               width=half_width, label_order=label_order, scheme=scheme_join) & \
                     get_chart(df_plot, bench_thesis, cpu=['HP'], metric='Time', wBlock=[8], mode=modes_nopenalty, domain=time_scale,
                               width=half_width, label_order=label_order, scheme=scheme_join)
ct_thesis_nopen = ct_thesis_a76_nopen | ct_thesis_hp_nopen

ct_thesis_a76_pen = get_chart(df_plot, bench_thesis, cpu=['A76'],metric='Dynamic', wBlock=[8], mode=modes_penalty, domain=energy_scale,
                              width=half_width, label_order=label_order, scheme=scheme_join) & \
                    get_chart(df_plot, bench_thesis, cpu=['A76'], metric='Leakage', wBlock=[8], mode=modes_penalty, domain=energy_scale,
                              width=half_width, label_order=label_order, scheme=scheme_join) & \
                    get_chart(df_plot, bench_thesis, cpu=['A76'], metric='Total', wBlock=[8], mode=modes_penalty, domain=energy_scale,
                              width=half_width, label_order=label_order, scheme=scheme_join) & \
                    get_chart(df_plot, bench_thesis, cpu=['A76'], metric='Time', wBlock=[8], mode=modes_penalty, domain=time_scale,
                              width=half_width, label_order=label_order, scheme=scheme_join)
ct_thesis_hp_pen = get_chart(df_plot, bench_thesis, cpu=['HP'], metric='Dynamic', wBlock=[8], mode=modes_penalty, domain=energy_scale,
                             width=half_width, label_order=label_order, scheme=scheme_join) & \
                   get_chart(df_plot, bench_thesis, cpu=['HP'], metric='Leakage', wBlock=[8], mode=modes_penalty, domain=energy_scale,
                             width=half_width, label_order=label_order, scheme=scheme_join) & \
                   get_chart(df_plot, bench_thesis, cpu=['HP'], metric='Total', wBlock=[8], mode=modes_penalty, domain=energy_scale,
                             width=half_width, label_order=label_order, scheme=scheme_join) & \
                   get_chart(df_plot, bench_thesis, cpu=['HP'], metric='Time', wBlock=[8], mode=modes_penalty, domain=time_scale,
                             width=half_width, label_order=label_order, scheme=scheme_join)
ct_thesis_pen = ct_thesis_a76_pen | ct_thesis_hp_pen

In [ ]:
# Plots paper.

scheme_a76 = ['#ffaa73', '#a84400', '#95baff', '#005dff', '#84cd84', '#007700']
scheme_hp = ['#ffc9a5', '#ff8533', '#8c3800', '#c5d2e9', '#5ea1ff', '#004cd2', '#9ae59a', '#3abf3a', '#006000']
scheme_join = scheme_a76 + scheme_hp
half_width = 400

ct_paper_a76 = get_chart(df_plot, bench_paper, cpu=['A76'],metric='Dynamic', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                         width=half_width, label_order=label_order, scheme=scheme_join) & \
               get_chart(df_plot, bench_paper, cpu=['A76'], metric='Leakage', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                         width=half_width, label_order=label_order, scheme=scheme_join) & \
               get_chart(df_plot, bench_paper, cpu=['A76'], metric='Total', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                         width=half_width, label_order=label_order, scheme=scheme_join) & \
               get_chart(df_plot, bench_paper, cpu=['A76'], metric='Time', wBlock=[8], mode=modes_nopenalty, domain=time_scale,
                         width=half_width, label_order=label_order, scheme=scheme_join)
ct_paper_hp = get_chart(df_plot, bench_paper, cpu=['HP'], metric='Dynamic', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                        width=half_width, label_order=label_order, scheme=scheme_join) & \
              get_chart(df_plot, bench_paper, cpu=['HP'], metric='Leakage', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                        width=half_width, label_order=label_order, scheme=scheme_join) & \
              get_chart(df_plot, bench_paper, cpu=['HP'], metric='Total', wBlock=[8], mode=modes_nopenalty, domain=energy_scale,
                        width=half_width, label_order=label_order, scheme=scheme_join) & \
              get_chart(df_plot, bench_paper, cpu=['HP'], metric='Time', wBlock=[8], mode=modes_nopenalty, domain=time_scale,
                        width=half_width, label_order=label_order, scheme=scheme_join)
ct_paper = ct_paper_a76 | ct_paper_hp
#ct_paper

In [ ]:
# Penalty plot thesis.

half_width  = 400
half_heigth = 100
labelXAngle = -30

df_penalty = df_plot.copy()
df_penalty['Mode'] = df_penalty['Mode'].replace({'PackingPenalty':'Packing', 'FusingPenalty':'Fusing'})
bench = list(bench_thesis.keys())

scheme = 2 * ['#95baff', '#005dff', '#84cd84', '#007700']
ct_penalty = (get_chart(df_penalty, bench, cpu=['A76'], metric='Total', wBlock=[8], fu=[2], mode=modes_mechanisms,
              domain=energy_scale, width=half_width, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
              get_chart(df_penalty, bench, cpu=['A76'], metric='Time', wBlock=[8], fu=[2], mode=modes_mechanisms,
              domain=half_time_scale, width=half_width, height=half_heigth, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle)) | \
             (get_chart(df_penalty, bench, cpu=['HP'],metric='Total', wBlock=[8], fu=[3], mode=modes_mechanisms,
              domain=energy_scale, width=half_width, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle) & \
              get_chart(df_penalty, bench, cpu=['HP'], metric='Time', wBlock=[8], fu=[3], mode=modes_mechanisms,
              domain=half_time_scale, width=half_width, height=half_heigth, label_order=label_order, scheme=scheme, labelXAngle=labelXAngle))
configure_chart(ct_penalty, orient='right')

In [ ]:
# Penalty plot paper.

half_width  = 400
half_heigth = 100

df_penalty = df_plot.copy()
df_penalty['Mode'] = df_penalty['Mode'].replace({'PackingPenalty':'Packing', 'FusingPenalty':'Fusing'})

scheme = 2 * ['#95baff', '#005dff', '#84cd84', '#007700']
ct_penalty_paper = (get_chart(df_penalty, bench_paper, cpu=['A76'], metric='Dynamic', wBlock=[8], fu=[2], mode=modes_mechanisms,
              domain=energy_scale, width=half_width, label_order=label_order, scheme=scheme) & \
              get_chart(df_penalty, bench_paper, cpu=['A76'], metric='Time', wBlock=[8], fu=[2], mode=modes_mechanisms,
              domain=half_time_scale, width=half_width, height=half_heigth, label_order=label_order, scheme=scheme)) | \
             (get_chart(df_penalty, bench_paper, cpu=['HP'],metric='Dynamic', wBlock=[8], fu=[3], mode=modes_mechanisms,
              domain=energy_scale, width=half_width, label_order=label_order, scheme=scheme) & \
              get_chart(df_penalty, bench_paper, cpu=['HP'], metric='Time', wBlock=[8], fu=[3], mode=modes_mechanisms,
              domain=half_time_scale, width=half_width, height=half_heigth, label_order=label_order, scheme=scheme))
ct_penalty_paper

In [ ]:
# Add wblock config column.

df_wblock = df_energy_time.reset_index().copy()
df_wblock['Label'] = df_plot['Benchmark']
df_wblock

In [ ]:
wblock_bench_paper = ['gemm', 'median', 'cartoon', 'average-paper']
half_width = 250
ct_wblock_paper = get_chart(df_wblock, wblock_bench_paper, view='wblock', cpu=['HP'], metric='Dynamic', mode=['Fusing'], fu=[3], domain=energy_scale,
                            width=half_width, label='Benchmark', label_order=wblock_bench_paper) | \
                  get_chart(df_wblock, wblock_bench_paper, view='wblock', cpu=['HP'], metric='Total', mode=['Fusing'], fu=[3], domain=energy_scale,
                            width=half_width, label='Benchmark', label_order=wblock_bench_paper)
ct_wblock_paper = configure_chart(ct_wblock_paper, orient='bottom')
ct_wblock_paper

In [ ]:
# Save plots as svg.

if make_plots:
    # Save A76 diffenergy plots.
    ct_nopen_A76.save(str(plot_dir / 'a76_diffenergy_time_nopen.svg'), webdriver='firefox')
    ct_pen_A76.save(str(plot_dir / 'a76_diffenergy_time_pen.svg'), webdriver='firefox')

    # Save HP diffenergy plots.
    ct_nopen_HP.save(str(plot_dir / 'hp_diffenergy_time_nopen.svg'), webdriver='firefox')
    ct_pen_HP.save(str(plot_dir / 'hp_diffenergy_time_pen.svg'), webdriver='firefox')

    # Save latency penalty plot.
    ct_penalty.save(str(plot_dir / 'penalty_compare_thesis.svg'), webdriver='firefox')
    ct_penalty_paper.save(str(plot_dir / 'penalty_compare_paper.svg'), webdriver='firefox')

    # Save thesis diffenergy plots.
    ct_thesis_nopen.save(str(plot_dir / 'diffenergy_time_nopen_thesis.svg'), webdriver='firefox')
    ct_thesis_pen.save(str(plot_dir / 'diffenergy_time_pen_thesis.svg'), webdriver='firefox')

    # Save paper plot diffenergy plot.
    ct_paper.save(str(plot_dir / 'diffenergy_time_nopen_paper.svg'), webdriver='firefox')

    # Save wblock plots.
    ct_wblock_paper.save(str(plot_dir / 'wblock_comparison_paper.svg'), webdriver='firefox')